# Drought and Wildfire Conditions

## Drought Conditions

![banner](img/coq_drawdown.jpg)

### Local Drought Levels

The table below shows drought levels in the Lower Mainland basin for 2015 to 2024. Drought levels reached level 3 on July 17th this year, and remained at this level until the end of September. In level 3, adverse impacts are possible. The lower mainland has experienced significant drought for the past four summers.

In [1]:
#Import libraries
import os
import pandas as pd
import subprocess
import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import json
from pandas import json_normalize

In [2]:
os.chdir('C://Users/pmarshal/Documents/Climate-Summary')
os.getcwd()

'C:\\Users\\pmarshal\\Documents\\Climate-Summary'

In [3]:
drought = pd.read_csv('data/drought.csv') 
drought['date'] = pd.to_datetime(dict(year=drought.year, month=drought.month, day=drought.day))

In [4]:
colorscale = alt.Scale(domain=['-0.1', '0', '1', '2', '3', '4', '5'],
                       range=['#1C00ff00', 'lightgreen', 'yellow', 'orange', '#E17F03', 'red', 'darkred'])

alt.Chart(drought, title="Drought Levels - Lower Mainland Basin").mark_square(size=300).encode(
    alt.X("monthdate(date):T", title=None),
    alt.Y("year:O", title=None),
    alt.Color("level:Q", title="Drought Level", scale=colorscale),
    tooltip=[
        alt.Tooltip("monthdate(date)", title="Date"),
        alt.Tooltip("level", title="Drought Level"),
    ],
).configure_view(
    step=14,
    strokeWidth=0
).configure_axis(
    domain=False
).properties(width=600, height=200)

alt.Chart(...)

In [5]:
spei = pd.read_csv('data/spi.csv', parse_dates=['date'])
spei = spei.round(2) 


### Standardized Precipitation Evapotranspiration Index

The Standardized Precipitation Evapotranspiration Index (SPEI) is a multi-scalar drought monitoring index based on climatic data. It's useful in determining the onset, duration, and magnitude of drought conditions, and captures the main impact of increased temperatures on water demand. 

For this report, the SPEI is calculated over 3-month intervals. This is useful for analyzing seasonal drought in the Lower Mainland. Values that fall between 1.0 and -1.0 are considered normal, values below -1.0 highlight dry conditions (red), and values above 1.0 are wet (blue). Each bar in the plot below represents the SPEI value for the previous 3 months. As we can see, conditions have been dry-to-very dry for the past two summers. We'll examine this drought more closely in the next section.

In [6]:
horz_line1 = alt.Chart(pd.DataFrame({
  'spei_3': [1],
  'color': ['black']
})).mark_rule(opacity=0.8, strokeDash=[3,3]).encode(
  y='spei_3',
  color=alt.Color('color:N', scale=None)
)

horz_line2 = alt.Chart(pd.DataFrame({
  'spei_3': [-1],
  'color': ['black']
})).mark_rule(opacity=0.8, strokeDash=[3,3]).encode(
  y='spei_3',
  color=alt.Color('color:N', scale=None)
)

title = alt.TitleParams(
   text='Standardized Precipitation Evapotranspiration Index',
   subtitle="Lower Capilano Watershed",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

bar = alt.Chart(spei, title=title).mark_bar(size=10).encode(
    alt.X("date:T", title=None, axis=alt.Axis(format='%b-%y')),
    alt.Y("spei_3:Q", title="SPEI 3-month"),
    color=alt.condition(
        alt.datum.spei_3 > 0,
        alt.value("blue"),  # The positive color
        alt.value("red")  # The negative color
    ),
    tooltip=[alt.Tooltip('date', title="Date"), alt.Tooltip('spei_3', title="SPEI-3 month")]
).properties(width=600)

bar + horz_line1 + horz_line2

alt.LayerChart(...)

In [7]:
fires = pd.read_csv('data/fire_stats.csv', parse_dates= ['date'])
fires['year'] = pd.DatetimeIndex(fires['date']).year

In [8]:
danger = pd.read_csv('data/seyfw_danger_filter.csv', parse_dates= ['datetime']) 
danger['year'] = pd.DatetimeIndex(danger['datetime']).year
danger['month'] = pd.DatetimeIndex(danger['datetime']).month
danger['DOY'] = pd.DatetimeIndex(danger['datetime']).dayofyear
danger['week'] = danger['datetime'].dt.isocalendar().week
#danger.head()

In [9]:
danger_filter = danger.loc[(danger['month']>=4)]
danger_filter = danger_filter.loc[(danger_filter['month']<=10)]
#danger_filter.head(12)
#danger_filter.to_csv('danger_filter.csv', index=False)

In [10]:
danger_count  = danger_filter.loc[(danger_filter['DGR']>=1)]
df_dgr = danger_count.loc[:,'year':'datetime']
#danger_count = danger_count.groupby(['year', 'DGR']).count().reset_index()
df_dgr = df_dgr.rename(columns={"year":"year", "datetime": "fire_danger"})
#danger_count['moving_avg'] = danger_count['DGR'].rolling(10).mean()
#danger_count

## Wildfire Conditions
### Provincial Wildfire Conditions
Provincially, 2024 was another challenging wildfire season. The majority of fires and area burned occurred in the northern part of the province, particularly in the northeast. It was a relatively quiet season in the Coastal Fire Centre. 

Below you can see current BC Wildfire Statistics in comparison to past seasons. All fire zones in the province have seen significant fire activity this season. Source: [BCWS](https://www2.gov.bc.ca/gov/content/safety/wildfire-status/about-bcws/wildfire-statistics/wildfire-averages#:~:text=Wildfire%20Averages%20%20%20%20Year%20%20,%20375%20%2845%25%29%20%2012%20more%20rows%20).

In [11]:
#Area burned chart

bar = alt.Chart(fires).mark_bar(opacity=0.8, color='darkgray', size = 10).encode(
    alt.X('date:T', title=None),
    alt.Y('area_burned:Q', title='Area Burned in Hectares', axis=alt.Axis(format='s')),
    color=alt.condition(
        alt.datum.year == 2024,  # If the country is "US" this test returns True,
        alt.value('red'),     # highlight a bar with red.
        alt.value('darkgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('area_burned', title="Area Burned")]
).properties(width=350, height=250, title='BC Area Burned per Year')


bar2 = alt.Chart(fires).mark_bar(opacity=0.8, color='red', size = 10).encode(
    alt.X('date:T', title=None),
    alt.Y('fire_no:Q', title='Number of Fires', axis=alt.Axis(format='s')),
    color=alt.condition(
        alt.datum.year == 2024,  # If the country is "US" this test returns True,
        alt.value('red'),     # highlight a bar with red.
        alt.value('darkgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('fire_no', title="Number of Fires")]
).properties(width=350, height=250, title='BC Number of Fires')


bar | bar2


alt.HConcatChart(...)

### Watershed Fire Danger

The plot below shows the number of days in each fire danger rating in the Seymour WSA. Scroll over each colour to see the number of days in each danger rating per season. 

In [12]:
title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Days in each fire danger rating (Apr 1 - Oct 31)",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

#domain = [5, 4, 3, 2, 1]  # Reversed domain
#range_ = ['darkred', '#FE3F46', 'orange', 'green', 'blue']  # Reversed range
domain = ['1 - Very Low', '2 - Low', '3 - Moderate', '4 - High', '5 - Extreme']
range_ = ['blue', 'green', 'yellow', 'darkorange', 'red']

test2 = alt.Chart(danger_filter, title=title).mark_bar(opacity=0.7).encode(
    alt.X('year:O', title=None),
    alt.Y('count(DGR):Q', title = 'Total days', sort='descending'),
    alt.Color("rating:O", title='Danger Rating', scale=alt.Scale(domain=domain, range=range_)),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('count(DGR)', title="Number of Days")]
)

rule2 = alt.Chart(df_dgr).mark_line(color='black', opacity=0.7, strokeDash=[4, 2]).encode(
    alt.X('year:O', title=None),
    alt.Y('moving_avg:Q', title= 'Total days')
)


test2.properties(width=600)

alt.Chart(...)

In [13]:
# Import data for crossover conditions and format dates
df = pd.read_csv('data/seyfw_crossover.csv', parse_dates= ['datetime']) 
df['year'] = pd.DatetimeIndex(df['datetime']).year
df['month'] = pd.DatetimeIndex(df['datetime']).month
df['DOY'] = pd.DatetimeIndex(df['datetime']).dayofyear

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7502 entries, 0 to 7501
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  7502 non-null   datetime64[ns]
 1   temp      7502 non-null   float64       
 2   rh        7502 non-null   int64         
 3   year      7502 non-null   int64         
 4   month     7502 non-null   int64         
 5   DOY       7502 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 351.8 KB


In [14]:
# Filter dataframe for crossover days
df_filter = df.loc[(df['temp']>=30) & (df['rh'] <= 30)]

In [15]:
# Count crossover days per year
df_count = df_filter.groupby('year').count().reset_index()

In [16]:
# Filter for one column and change the name of the column
df_co = df_count.loc[:,'year':'datetime']
df_co = df_co.rename(columns={"year":"year", "datetime": "crossover_count"})
df_co['moving_avg'] = df_co['crossover_count'].rolling(10).mean()

The plot below shows the number of days where 30/30 crossover conditions were experienced each year. 30/30 crossover refers to days where temperatures exceed 30 degrees Celsius and relative humidity is lower than 30%. These are very hot and dry days! During these conditions, if there is a fire, it's likely to see explosive fire behaviour. The red dashed line shows the 10-year moving average. In the past twenty years the number of days with crossover conditions has increase from around 4 to 10; however, there were fewer crossover days this past year. 

In [17]:
title = alt.TitleParams(
   text='Seymour Watershed',
   subtitle="Days in crossover condition (30/30) per year",
   anchor='middle',
   fontSize=14,
   fontWeight='bold')

bar = alt.Chart(df_co, title=title).mark_bar().encode(
    alt.X('year:O', title=None),
    alt.Y('crossover_count:Q', title = 'Count of Days'),
    color=alt.condition(
        alt.datum.year == 2024,  # If the country is "US" this test returns True,
        alt.value('blue'),     # highlight a bar with red.
        alt.value('lightgrey')   # And grey for the rest of the bars
     ),
    tooltip=[alt.Tooltip('year', title="Year"), alt.Tooltip('crossover_count', title="Number of Days")]
)

rule = alt.Chart(df_co).mark_line(color='red', strokeDash=[4, 2]).encode(
    alt.X('year:O', title=None),
    alt.Y('moving_avg:Q', title= 'Count of Days')
)

(bar + rule).properties(width=600)

alt.LayerChart(...)